### Second: provide SQL queries
Answer three of the following questions with at least one question coming from the closed-ended and one from the open-ended question set. Each question should be answered using one query.
Closed-ended questions:
* What are the top 5 brands by receipts scanned among users 21 and over?
    1. 'NERDS CANDY'
    2. 'DOVE'
    3. 'TRIDENT'
    4. 'SOUR PATCH KIDS'
    5. 'MEIJER'
* What are the top 5 brands by sales among users that have had their account for at least six months?
    1. 'CVS'
    2. 'DOVE'
    3. 'TRIDENT'
    4. 'COORS LIGHT'
    5. 'TRESEMMÉ'
3. What is the percentage of sales in the Health & Wellness category by generation?
    * 'Zoomers', 0.0
    * 'Millenials', 19.43
    * 'X',16.88
    * 'Boomers', 27.66
    * 'Silent', 0.0
    * 'Other', 24.16 - missing birth_date

Open-ended questions: for these, make assumptions and clearly state them when answering the question.
1. Who are Fetch’s power users?
2. Which is the leading brand in the Dips & Salsa category? - Tostitos*
    * Long answer: The dataset here is incomplete, as the transactions are missing a substantial amount of barcodes. Based on the barcodes present, Tostitos is the leading brand by 3 metrics - number of sales, sum of sales in $, and number of receipts. This also lines up with public sales records, such as an older report from Statista, as well as personal biases when browsing this category in stores. https://www.statista.com/statistics/300313/us-leading-salsa-based-on-dollar-sales/#:~:text=Tostitos%20is%20the%20leading%20brand,owned%20and%20operated%20by%20PepsiCo.
3. At what percent has Fetch grown year over year? - not enough data, only 4 months of 1 year is available

### Importing packages and data into environment

In [3]:
import sqlite3
import pandas as pd

# Connect to SQLite (create the database file if it doesn't exist)
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# List of CSV file paths and corresponding table names
csv_files = [
    ('USER_TAKEHOME.csv', 'users'),
    ('PRODUCTS_TAKEHOME.csv', 'products'),
    ('transactions_updated.csv', 'transactions')
]

# Function to create a table and import CSV data
def import_csv_to_sql(csv_file, table_name):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Create the SQL table if it doesn't exist, using the DataFrame columns as SQL columns
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Data from {csv_file} imported into {table_name}")

# Loop through the list of CSV files and import each one
for csv_file, table_name in csv_files:
    import_csv_to_sql(csv_file, table_name)

# Commit and close the connection
conn.commit()
conn.close()

Data from USER_TAKEHOME.csv imported into users
Data from PRODUCTS_TAKEHOME.csv imported into products
Data from transactions_updated.csv imported into transactions


### What are the top 5 brands by receipts scanned among users 21 and over?
1. 'NERDS CANDY'
2. 'DOVE'
3. 'TRIDENT'
4. 'SOUR PATCH KIDS'
5. 'MEIJER'

In [5]:
# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select p.brand, 
       count(distinct(t.receipt_id)) as receipts
from transactions t
left join products p
    on t.barcode = p.barcode
where user_id in (
    SELECT ID
    FROM users 
    WHERE BIRTH_DATE < DATE('now','-21 years') 
)
and brand <> 'None'  -- result depends on whether you want to include/exclude 'None'
group by p.brand
order by receipts desc
limit 5
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('TRIDENT', 2)
('SOUR PATCH KIDS', 2)
('NERDS CANDY', 2)
('DOVE', 2)
('UTZ', 1)


### 2. What are the top 5 brands by sales among users that have had their account for at least six months?
    1. 'CVS'
    2. 'DOVE'
    3. 'TRIDENT'
    4. 'COORS LIGHT'
    5. 'TRESEMMÉ'

In [7]:
# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select p.brand, count(distinct(t.receipt_id)), sum(final_sale)
from transactions t
left join products p
    on t.barcode = p.barcode
where user_id in (
    SELECT ID
    FROM users 
    WHERE CREATED_DATE < DATE('now','-6 months') 
)
and brand <> 'None' --result depends on if you want to include / exclude 'None'
group by 1
order by 3 desc
LIMIT 5
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('TRIDENT', 2, 23.36)
('DOVE', 2, 18.94)
('COORS LIGHT', 1, 17.48)
('TRESEMMÉ', 1, 14.58)
('PEPPERIDGE FARM', 1, 12.15)


### 3. What is the percentage of sales in the Health & Wellness category by [age] generation?
* 'Zoomers', 0.0
* 'Millenials', 19.43
* 'X',16.88
* 'Boomers', 27.66
* 'Silent', 0.0
* 'Other', 24.16 - missing birth_date

In [9]:
# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select case when birth_date >= '2022-01-01' then '8.Alpha'
    when birth_date >= '1997-01-01' then '7.Zoomers'
    when birth_date >= '1981-01-01' then '6.Millenials'
    when birth_date >= '1965-01-01' then '5.X'
    when birth_date >= '1946-01-01' then '4.Boomers'
    when birth_date >= '1928-01-01' then '3.Silent'
    when birth_date >= '1901-01-01' then '2.Greatest'
    when birth_date >= '1883-01-01' then '1.Lost'
    else 'Other'
    end as generation,
--sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end), 
--sum(final_sale),
round(100.0*sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end) / sum(final_sale),2) as health_perc
from transactions t
left join products p
    on t.barcode = p.barcode
left join users u
    on u.id = t.user_id
group by 1
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('3.Silent', 0.0)
('4.Boomers', 3.57)
('5.X', 24.15)
('6.Millenials', 17.47)
('Other', 26.22)


In [10]:
# Confirming that 'Other' lines up with missing birth_dates

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
    sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end), 
    sum(final_sale),
    round(100.0*sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end) / sum(final_sale),4) as health_perc
from transactions t
left join products p
    on t.barcode = p.barcode
left join users u
    on u.id = t.user_id
where birth_date is null
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(13918.78, 53089.96, 26.2173)


### 2.2. Which is the leading brand in the Dips & Salsa category?
Tostitos
* Long answer: The dataset here is incomplete, as the transactions are missing a substantial amount of barcodes. Based on the barcodes present, Tostitos is the leading brand by 3 metrics - number of sales, sum of sales in $, and number of receipts. This also lines up with public sales records, such as an older report from Statista, as well as personal biases when browsing this category in stores. https://www.statista.com/statistics/300313/us-leading-salsa-based-on-dollar-sales/#:~:text=Tostitos%20is%20the%20leading%20brand,owned%20and%20operated%20by%20PepsiCo.

In [12]:
# View leading brands by number of sales from customer receipts in our data

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
brand, count(distinct(receipt_id)), sum(final_quantity), round(sum(final_sale),2)
from transactions t
left join products p
    on t.barcode = p.barcode
where category_2 = 'Dips & Salsa'
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('TOSTITOS', 21, 22.0, 97.34)
('PACE', 13, 13.0, 48.69)
('GOOD FOODS', 5, 5.0, 46.95)
('FRITOS', 12, 13.0, 43.55)
(None, 8, 8.0, 43.16)
('MARKETSIDE', 8, 8.0, 35.7)
('MARZETTI', 7, 7.0, 32.05)
('HELUVA GOOD!', 7, 7.0, 25.26)
('KIRKLAND SIGNATURE', 2, 2.0, 23.48)
('HIDDEN VALLEY', 6, 12.0, 23.14)


In [13]:
# View number of products in dataset

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select brand, count(1)
from products p
where category_2 = 'Dips & Salsa'
group by 1
order by 2 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(None, 6329)
('PRIVATE LABEL', 1166)
('SABRA', 268)
('WHOLLY', 217)
('TOSTITOS', 149)
('PACE', 119)
('GOOD FOODS', 115)
('MEIJER', 112)
('MARZETTI', 109)
('BRAND NOT KNOWN', 108)


### Miscellaneous tests

In [15]:
# Missing user data - check to confirm that most user_ids are missing from user table when joining

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
count(distinct(case when u.id is null then user_id end)) as missing_userid,
count(distinct(user_id)) as total_users
from transactions t
left join users u
    on u.id = t.user_id
limit 100
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(10143, 10190)


In [16]:
# Missing user data - check to confirm that most user_ids are missing from user table when joining

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
min(birth_date), max(birth_date)
from transactions t
left join users u
    on t.user_id = u.id
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('1943-09-03 05:00:00.000 Z', '1996-05-20 00:00:00.000 Z')


### Exploration
These queries are irrelevant to the questions asked, but are for personal exploration, and identifying trends in the data.

In [18]:
# Check most popular stores

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select store_name, count(distinct(receipt_id)), sum(final_quantity), sum(final_sale),
round(100.0*round(sum(final_sale),2) / (select sum(final_sale) from transactions),2) as perc_of_sales
from transactions
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

# Check most popular stores

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select category_1, count(1), sum(final_quantity), round(sum(final_sale),2),
round(100.0*round(sum(final_sale),2) / (select sum(final_sale) from transactions),2) as perc_of_sales
from products p
left join transactions t
    on p.barcode = t.barcode
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('WALMART', 5176, 5585.99, 20777.59, 38.93)
('COSTCO', 255, 243.0, 3261.19, 6.11)
('TARGET', 385, 434.0, 2549.11, 4.78)
("SAM'S CLUB", 234, 226.0, 2484.94, 4.66)
('DOLLAR GENERAL STORE', 674, 736.0, 2226.76, 4.17)
('WALGREENS', 259, 281.0, 1994.33, 3.74)
('CVS', 308, 244.0, 1298.07, 2.43)
('MEIJER', 292, 363.22, 1287.58, 2.41)
('KROGER', 359, 349.0, 1205.38, 2.26)
('AMAZON', 77, 90.0, 1099.34, 2.06)
('Snacks', 326277, 3961.86, 15242.6, 28.56)
('Health & Wellness', 512971, 1785.0, 13960.05, 26.16)
('Beverages', 4284, 508.0, 2363.11, 4.43)
(None, 290, 195.47, 866.21, 1.62)
('Alcohol', 509, 27.0, 365.0, 0.68)
('Pantry', 891, 62.17, 247.02, 0.46)
('Dairy', 619, 49.0, 167.84, 0.31)
('Deli & Bakery', 71, 9.0, 53.95, 0.1)
('Frozen', 62, 4.0, 25.0, 0.05)
('Needs Review', 547, 2.0, 1.45, 0.0)


In [19]:
# Check most popular stores

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select category_3, count(1), sum(final_quantity), round(sum(final_sale),2)
from products p
left join transactions t
    on p.barcode = t.barcode
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(None, 60899, 692.16, 3512.76)
('Cola', 2794, 443.0, 2201.66)
('Chocolate Candy', 47956, 570.0, 1702.94)
('Vitamins & Herbal Supplements', 55708, 117.0, 1560.39)
('Confection Candy', 57074, 382.55, 1071.98)
('Potato Chips', 11336, 301.0, 1037.95)
('Tortilla Chips', 4049, 203.0, 784.06)
('Body Wash', 15092, 94.0, 746.7)
('Allergy & Sinus Medicines & Treatments', 5798, 47.0, 696.16)
('Digestive Health Medicines & Treatments', 9603, 71.0, 574.34)


In [20]:
# Missing user data - check to confirm that most user_ids are missing from user table when joining
# transactions contains 17,694 ids, users contains 100,000, only 91 show up in both

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
SELECT
    -- Count distinct user_id not in users table
    (SELECT COUNT(DISTINCT t.user_id)
     FROM transactions t
     LEFT JOIN users u ON t.user_id = u.id
     WHERE u.id IS NULL) AS t_notin_u,

    -- Count distinct id not in transactions table
    (SELECT COUNT(DISTINCT u.id)
     FROM users u
     LEFT JOIN transactions t ON u.id = t.user_id
     WHERE t.user_id IS NULL) AS u_notin_t,

    -- Count distinct user_id in transactions table
    (SELECT COUNT(DISTINCT user_id)
     FROM transactions) AS trxn_ids,

    -- Count distinct id in users table
    (SELECT COUNT(DISTINCT id)
     FROM users) AS u_ids
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(10143, 99953, 10190, 100000)
